In [8]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [9]:
import tensorflow_model_optimization as tfmot
import tensorflow as tf
from tensorflow import keras
import zipfile
from tensorflow.keras import   datasets
from tensorflow.keras.optimizers     import   SGD, Adam
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, AveragePooling2D, MaxPooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers

In [3]:
LastValueQuantizer = tfmot.quantization.keras.quantizers.LastValueQuantizer
MovingAverageQuantizer = tfmot.quantization.keras.quantizers.MovingAverageQuantizer
annotate = tfmot.quantization.keras.quantize_annotate_layer


class ModifiedDenseQuantizeConfig(tfmot.quantization.keras.QuantizeConfig):
    def get_weights_and_quantizers(self, layer):
        return [(layer.kernel, LastValueQuantizer(num_bits=16, symmetric=True, narrow_range=False, per_axis=False))]

    def get_activations_and_quantizers(self, layer):
        return [(layer.activation, MovingAverageQuantizer(num_bits=16, symmetric=False, narrow_range=False, per_axis=False))]

    def set_quantize_weights(self, layer, quantize_weights):
      # Add this line for each item returned in `get_weights_and_quantizers`
      # , in the same order
        layer.kernel = quantize_weights[0]

    def set_quantize_activations(self, layer, quantize_activations):
      # Add this line for each item returned in `get_activations_and_quantizers`
      # , in the same order.
        layer.activation = quantize_activations[0]

    # Configure how to quantize outputs (may be equivalent to activations).
    def get_output_quantizers(self, layer):
        return []

    def get_config(self):
        return {}


In [4]:
class activationquant(tfmot.quantization.keras.QuantizeConfig):
    
    def _assert_activation_layer(self, layer):
        if not isinstance(layer, tf.keras.layers.Activation):
            raise RuntimeError(
          'Default8BitActivationQuantizeConfig can only be used with '
          '`keras.layers.Activation`.')

    def get_weights_and_quantizers(self, layer):
        self._assert_activation_layer(layer)
        return []

    def get_activations_and_quantizers(self, layer):
        self._assert_activation_layer(layer)
        return []

    def set_quantize_weights(self, layer, quantize_weights):
        self._assert_activation_layer(layer)

    def set_quantize_activations(self, layer, quantize_activations):
        self._assert_activation_layer(layer)

    def get_output_quantizers(self, layer):
        self._assert_activation_layer(layer)

        if not hasattr(layer.activation, '__name__'):
            raise ValueError('Activation {} not supported by '
                       'Default8BitActivationQuantizeConfig.'.format(
                           layer.activation))

        if layer.activation.__name__ in ['relu', 'swish']:
          # 'relu' should generally get fused into the previous layer.
          return [tfmot.quantization.keras.quantizers.MovingAverageQuantizer(
              num_bits=16, per_axis=False, symmetric=False, narrow_range=False)]
        elif layer.activation.__name__ in [
            'linear', 'softmax', 'sigmoid', 'tanh']:
            return []

        raise ValueError('Activation {} not supported by '
                         'Default8BitActivationQuantizeConfig.'.format(
                             layer.activation))

    def get_config(self):
        return {}

## MobilenetV1

In [5]:
# for dephwise conv issue: https://stackoverflow.com/questions/48121702/float-ops-found-in-quantized-tensorflow-mobilenet-model


l2 = tf.keras.regularizers.L2

def mobilenet_v1():
    # Mobilenet parameters
    input_shape = [96,96,3] # resized to 96x96 per EEMBC requirement
    num_classes = 2 # person and non-person
    num_filters = 8 # normally 32, but running with alpha=.25 per EEMBC requirement

    inputs = Input(shape=input_shape)
    x = inputs # Keras model uses ZeroPadding2D()

    # 1st layer, pure conv
    # Keras 2.2 model has padding='valid' and disables bias
    x = Conv2D(num_filters,
                  kernel_size=3,
                  strides=2,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x) # Keras uses ReLU6 instead of pure ReLU

    # 2nd layer, depthwise separable conv
    # Filter size is always doubled before the pointwise conv
    # Keras uses ZeroPadding2D() and padding='valid'
    x = DepthwiseConv2D(kernel_size=3,
                  strides=1,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    num_filters = 2*num_filters
    x = Conv2D(num_filters,
                  kernel_size=1,
                  strides=1,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # 3rd layer, depthwise separable conv
    x = DepthwiseConv2D(kernel_size=3,
                  strides=2,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    num_filters = 2*num_filters
    x = Conv2D(num_filters,
                  kernel_size=1,
                  strides=1,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # 4th layer, depthwise separable conv
    x = (DepthwiseConv2D(kernel_size=3,
                  strides=1,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4)))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(num_filters,
                  kernel_size=1,
                  strides=1,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # 5th layer, depthwise separable conv
    x = DepthwiseConv2D(kernel_size=3,
                  strides=2,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    num_filters = 2*num_filters
    x = Conv2D(num_filters,
                  kernel_size=1,
                  strides=1,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # 6th layer, depthwise separable conv
    x = DepthwiseConv2D(kernel_size=3,
                  strides=1,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(num_filters,
                  kernel_size=1,
                  strides=1,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # 7th layer, depthwise separable conv
    x = DepthwiseConv2D(kernel_size=3,
                  strides=2,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    num_filters = 2*num_filters
    x = Conv2D(num_filters,
                  kernel_size=1,
                  strides=1,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # 8th-12th layers, identical depthwise separable convs
    # 8th
    x = DepthwiseConv2D(kernel_size=3,
                  strides=1,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(num_filters,
                  kernel_size=1,
                  strides=1,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # 9th
    x = annotate(DepthwiseConv2D(kernel_size=3,
                  strides=1,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4)))(x)
    x = BatchNormalization()(x)
    x = annotate(Activation('relu'), quantize_config = activationquant())(x)

    x = annotate(Conv2D(num_filters,
                  kernel_size=1,
                  strides=1,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4)), quantize_config = ModifiedDenseQuantizeConfig())(x)
    x = BatchNormalization()(x)
    x = annotate(Activation('relu'), quantize_config = activationquant())(x)

    # 10th
    x = annotate(DepthwiseConv2D(kernel_size=3,
                  strides=1,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4)))(x)
    x = BatchNormalization()(x)
    x = annotate(Activation('relu'), quantize_config = activationquant())(x)

    x = annotate(Conv2D(num_filters,
                  kernel_size=1,
                  strides=1,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4)), quantize_config = ModifiedDenseQuantizeConfig())(x)
    x = BatchNormalization()(x)
    x = annotate(Activation('relu'), quantize_config = activationquant())(x)

    # 11th
    x = annotate(DepthwiseConv2D(kernel_size=3,
                  strides=1,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4)))(x)
    x = BatchNormalization()(x)
    x = annotate(Activation('relu'),quantize_config = activationquant())(x)

    x = annotate(Conv2D(num_filters,
                  kernel_size=1,
                  strides=1,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4)), quantize_config = ModifiedDenseQuantizeConfig())(x)
    x = BatchNormalization()(x)
    x = annotate(Activation('relu'), quantize_config = activationquant())(x)

    # 12th
    x = annotate(DepthwiseConv2D(kernel_size=3,
                  strides=1,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4)))(x)
    x = BatchNormalization()(x)
    x = annotate(Activation('relu'), quantize_config = activationquant())(x)

    x = annotate(Conv2D(num_filters,
                  kernel_size=1,
                  strides=1,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4)), quantize_config = ModifiedDenseQuantizeConfig())(x)
    x = BatchNormalization()(x)
    x = annotate(Activation('relu'), quantize_config = activationquant())(x)

    # 13th layer, depthwise separable conv
    x = annotate(DepthwiseConv2D(kernel_size=3,
                  strides=2,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4)))(x)
    x = BatchNormalization()(x)
    x = annotate(Activation('relu'), quantize_config = activationquant())(x)

    num_filters = 2*num_filters
    x = annotate(Conv2D(num_filters,
                  kernel_size=1,
                  strides=1,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4)), quantize_config = ModifiedDenseQuantizeConfig())(x)
    x = BatchNormalization()(x)
    x = annotate(Activation('relu'), quantize_config = activationquant())(x)

    # 14th layer, depthwise separable conv
    x = annotate(DepthwiseConv2D(kernel_size=3,
                  strides=1,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4)))(x)
    x = BatchNormalization()(x)
    x = annotate(Activation('relu'), quantize_config = activationquant())(x)

    x = annotate(Conv2D(num_filters,
                  kernel_size=1,
                  strides=1,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4)), quantize_config = ModifiedDenseQuantizeConfig())(x)
    x = BatchNormalization()(x)
    x = annotate(Activation('relu'), quantize_config = activationquant())(x)

    # Average pooling, max polling may be used also
    # Keras employs GlobalAveragePooling2D 
    x = AveragePooling2D(pool_size=x.shape[1:3])(x)
    #x = MaxPooling2D(pool_size=x.shape[1:3])(x)

    # Keras inserts Dropout() and a pointwise Conv2D() here
    # We are staying with the paper base structure

    # Flatten, FC layer and classify
    x = Flatten()(x)
    outputs = annotate(Dense(num_classes, activation='softmax'), quantize_config = ModifiedDenseQuantizeConfig())(x)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [6]:
model = mobilenet_v1()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 96, 96, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 48, 48, 8)         224       
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 8)         32        
_________________________________________________________________
activation (Activation)      (None, 48, 48, 8)         0         
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 48, 48, 8)         80        
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 8)         32        
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 8)         0     

## Traninig

In [19]:
import os
import scipy


IMAGE_SIZE = 96
BATCH_SIZE = 32
EPOCHS = 20

BASE_DIR = os.path.join(os.getcwd(),'C:/Users/abirh/Desktop/codes/Actual_exp/MobilenetV1/p1/vw_coco2014_96')


def train_model():
    model = mobilenet_v1()
    batch_size = 50
    validation_split = 0.1

    quantize_scope = tfmot.quantization.keras.quantize_scope

    # `quantize_apply` requires mentioning `DefaultDenseQuantizeConfig` with `quantize_scope`
    # as well as the custom Keras layer.
    with quantize_scope({
        'ModifiedDenseQuantizeConfig':ModifiedDenseQuantizeConfig,
        'activationquant'   :activationquant}):
      # Use `quantize_apply` to actually make the model quantization aware.
      model = tfmot.quantization.keras.quantize_apply(model)

    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
          rotation_range=10,
          width_shift_range=0.05,
          height_shift_range=0.05,
          zoom_range=.1,
          horizontal_flip=True,
          validation_split=validation_split,
          rescale=1. / 255)
    train_generator = datagen.flow_from_directory(
          BASE_DIR,
          target_size=(IMAGE_SIZE, IMAGE_SIZE),
          batch_size=BATCH_SIZE,
          subset='training',
          color_mode='rgb')
    val_generator = datagen.flow_from_directory(
          BASE_DIR,
          target_size=(IMAGE_SIZE, IMAGE_SIZE),
          batch_size=BATCH_SIZE,
          subset='validation',
          color_mode='rgb')
    print(train_generator.class_indices)

    model = train_epochs(model, train_generator, val_generator, 20, 0.001)
    model = train_epochs(model, train_generator, val_generator, 10, 0.0005)
    model = train_epochs(model, train_generator, val_generator, 20, 0.00025)


def train_epochs(model, train_generator, val_generator, epoch_count,
                     learning_rate):
    model.compile(
          optimizer=tf.keras.optimizers.Adam(learning_rate),
          loss='categorical_crossentropy',
          metrics=['accuracy'])
    history_fine = model.fit(
          train_generator,
          steps_per_epoch=len(train_generator),
          epochs=epoch_count,
          validation_data=val_generator,
          validation_steps=len(val_generator),
          batch_size=BATCH_SIZE)
    return model

In [8]:
train_model() # Comment this once tranined and keep the traning results

Found 98658 images belonging to 2 classes.
Found 10961 images belonging to 2 classes.
{'non_person': 0, 'person': 1}
Epoch 1/20
3084/3084 [==============================] - 215s 68ms/step - loss: 0.9141 - accuracy: 0.6047 - val_loss: 0.8455 - val_accuracy: 0.6513
Epoch 2/20
3084/3084 [==============================] - 200s 65ms/step - loss: 0.7686 - accuracy: 0.6655 - val_loss: 0.7428 - val_accuracy: 0.6511
Epoch 3/20
3084/3084 [==============================] - 199s 65ms/step - loss: 0.6683 - accuracy: 0.6948 - val_loss: 0.6262 - val_accuracy: 0.7144
Epoch 4/20
3084/3084 [==============================] - 200s 65ms/step - loss: 0.6094 - accuracy: 0.7163 - val_loss: 0.5872 - val_accuracy: 0.7337
Epoch 5/20
3084/3084 [==============================] - 199s 64ms/step - loss: 0.5658 - accuracy: 0.7418 - val_loss: 0.5595 - val_accuracy: 0.7344
Epoch 6/20
3084/3084 [==============================] - 200s 65ms/step - loss: 0.5366 - accuracy: 0.7575 - val_loss: 0.5226 - val_accuracy: 0.7632
E

In [22]:
model.save('P1Q1_model') # Comment this once tranined

INFO:tensorflow:Assets written to: P1Q1_model/assets


INFO:tensorflow:Assets written to: P1Q1_model/assets


In [23]:
model.save('P1Q1_model.h5') # Comment this once tranined

## Loading model

In [11]:
model = tf.keras.models.load_model('P1Q1_model')

In [7]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
              rotation_range=10,
              width_shift_range=0.05,
              height_shift_range=0.05,
              zoom_range=.1,
              horizontal_flip=True,
              validation_split=.1,
              rescale=1. / 255)

val_generator = datagen.flow_from_directory(
              BASE_DIR,
              target_size=(IMAGE_SIZE, IMAGE_SIZE),
              batch_size=BATCH_SIZE,
              subset='validation',
              color_mode='rgb')

model.compile(
              metrics=['acc'])

acc = model.evaluate(val_generator)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\abirh\\Desktop\\codes\\Actual_exp\\MobilenetV1\\p1\\p1q1\\vw_coco2014_96'

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 96, 96, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 48, 48, 8)         224       
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 8)         32        
_________________________________________________________________
activation (Activation)      (None, 48, 48, 8)         0         
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 48, 48, 8)         80        
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 8)         32        
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 8)         0     

In [14]:
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
import os


# BASE_DIR = os.path.join(os.getcwd(),'vw_coco2014_96')
# new_dir = os.path.join(BASE_DIR, 'non_person/COCO_train2014_000000000009.jpg')
img = 'C:/Users/abirh/Desktop/codes/Actual_exp/MobilenetV1/p1/vw_coco2014_96/non_person/COCO_train2014_000000000009.jpg'
#load the image
my_image = load_img(img, target_size=(96, 96, 3))

#preprocess the image
my_image = img_to_array(my_image)
my_image = my_image.reshape((1, my_image.shape[0], my_image.shape[1], my_image.shape[2]))
my_image = preprocess_input(my_image)

#make the prediction
prediction = model.predict(my_image)
print("prediction index:",np.argmax(prediction, axis = -1)[0])
print("actual image: non-person")

prediction index: 0
actual image: non-person


## Partition MCU+Edge
firstPart = MCU, lastPart = Edge

In [15]:
# Partition Layer should be the last layer before the partition
def partition(model, partitionLayer):
  num_layers = len(model.layers) - 1
  if partitionLayer >= num_layers:
    return "pick a lower partition layer, yours is or is greater than the last layer"
  clientLayers = []
  serverLayers = []
  for i in range (0,partitionLayer+1):
      clientLayers.append(model.get_layer(index=i))
  for i in range (partitionLayer+1,len(model.layers)):
      serverLayers.append(model.get_layer(index=i))
  #Creating Client and Server models
  clientModel = tf.keras.Sequential(clientLayers)
  clientModel.add(Flatten())
  serverModel = tf.keras.Sequential(serverLayers)
  print("partitioning layer is =",partitionLayer)
  return clientModel, serverModel

In [16]:
partition_layer = 45
firstPart, lastPart = partition(model, partition_layer)

partitioning layer is = 45


## Run image through both partitions to create a summary

In [17]:
# from tensorflow.keras.applications.resnet50 import preprocess_input
# image_num = 0
#print(list(image.reshape(1,3072)[0]))
first_part_output = firstPart.predict(my_image)

new_shape = lastPart.layers[0].input_shape
first_part_output = first_part_output.reshape(1,new_shape[1], new_shape[2], new_shape[3])

last_part_output = lastPart.predict(first_part_output)
output = np.argmax(lastPart.predict(first_part_output))
print("actual image index: non-person")
print("predicted index:",output)
print(first_part_output.shape)
# print(first_part_output)

actual image index: non-person
predicted index: 0
(1, 6, 6, 128)


In [18]:
firstPart.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 8)         224       
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 8)         32        
_________________________________________________________________
activation (Activation)      (None, 48, 48, 8)         0         
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 48, 48, 8)         80        
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 8)         32        
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 16)        1

In [19]:
lastPart.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_annotate (QuantizeA (None, 6, 6, 128)         1280      
_________________________________________________________________
batch_normalization_15 (Batc (None, 6, 6, 128)         512       
_________________________________________________________________
quantize_annotate_1 (Quantiz (None, 6, 6, 128)         0         
_________________________________________________________________
quantize_annotate_2 (Quantiz (None, 6, 6, 128)         16512     
_________________________________________________________________
batch_normalization_16 (Batc (None, 6, 6, 128)         512       
_________________________________________________________________
quantize_annotate_3 (Quantiz (None, 6, 6, 128)         0         
_________________________________________________________________
quantize_annotate_4 (Quantiz (None, 6, 6, 128)        

## Deploy to MCU 
#### TFLite and .h model of First Part

In [20]:
def representative_dataset_gen():
        dataset_dir = os.path.join(BASE_DIR, "person")
        for idx, image_file in enumerate(os.listdir(dataset_dir)):
            # 10 representative images should be enough for calibration.
            if idx > 10:
                return
            full_path = os.path.join(dataset_dir, image_file)
            if os.path.isfile(full_path):
                img = tf.keras.preprocessing.image.load_img(
                    full_path, color_mode='rgb').resize((96, 96))
                arr = tf.keras.preprocessing.image.img_to_array(img)
                # Scale input to [0, 1.0] like in training.
                yield [arr.reshape(1, 96, 96, 3) / 255.]

converter = tf.lite.TFLiteConverter.from_keras_model(firstPart)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
tflite_model = converter.convert()
open("p1q1.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\abirh\AppData\Local\Temp\tmp8dt649qr\assets


INFO:tensorflow:Assets written to: C:\Users\abirh\AppData\Local\Temp\tmp8dt649qr\assets


68640

In [21]:
import binascii

def convert_to_c_array(bytes) -> str:
  hexstr = binascii.hexlify(bytes).decode("UTF-8")
  hexstr = hexstr.upper()
  array = ["0x" + hexstr[i:i + 2] for i in range(0, len(hexstr), 2)]
  array = [array[i:i+10] for i in range(0, len(array), 10)]
  return ",\n  ".join([", ".join(e) for e in array])

tflite_binary = open("p1q1.tflite", 'rb').read()
ascii_bytes = convert_to_c_array(tflite_binary)
c_file = "const unsigned char tf_model[] = {\n  " + ascii_bytes + "\n};\nunsigned int tf_model_len = " + str(len(tflite_binary)) + ";"
# print(c_file)
open("p1q1.h", "w").write(c_file)

425639